In [15]:
import sys
sys.path.append("/jmain02/home/J2AD007/txk47/cxz00-txk47/cliport")
sys.path.append("/jmain02/home/J2AD007/txk47/cxz00-txk47/cliport/mae")

In [20]:
import models_lib
import util.misc as misc
import torch

In [22]:
model = models_lib.__dict__['mae_robot_lang'](norm_pix_loss=False)

In [24]:
checkpoint_path = "/jmain02/home/J2AD007/txk47/cxz00-txk47/cliport/checkpoints/clip_vit_base_patch16.pth"
checkpoint_dict = torch.load(checkpoint_path)

/tmp/ipykernel_1141508/4050986575.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(checkpoint_path)


In [35]:
def print_checkpoint(state_dict):
    for key, value in state_dict.items():
         print(f'{key} : {value.shape}')

In [45]:
print_checkpoint(model.state_dict())

cls_token : torch.Size([1, 1, 768])
pos_embed : torch.Size([1, 201, 768])
mask_token : torch.Size([1, 1, 512])
decoder_pos_embed : torch.Size([1, 201, 512])
patch_embed.proj.weight : torch.Size([768, 3, 16, 16])
patch_embed.proj.bias : torch.Size([768])
blocks.0.norm1.weight : torch.Size([768])
blocks.0.norm1.bias : torch.Size([768])
blocks.0.attn.qkv.weight : torch.Size([2304, 768])
blocks.0.attn.qkv.bias : torch.Size([2304])
blocks.0.attn.proj.weight : torch.Size([768, 768])
blocks.0.attn.proj.bias : torch.Size([768])
blocks.0.norm2.weight : torch.Size([768])
blocks.0.norm2.bias : torch.Size([768])
blocks.0.mlp.fc1.weight : torch.Size([3072, 768])
blocks.0.mlp.fc1.bias : torch.Size([3072])
blocks.0.mlp.fc2.weight : torch.Size([768, 3072])
blocks.0.mlp.fc2.bias : torch.Size([768])
blocks.1.norm1.weight : torch.Size([768])
blocks.1.norm1.bias : torch.Size([768])
blocks.1.attn.qkv.weight : torch.Size([2304, 768])
blocks.1.attn.qkv.bias : torch.Size([2304])
blocks.1.attn.proj.weight : to

In [38]:
print_checkpoint(checkpoint_dict)

logit_scale : torch.Size([])
text_model.embeddings.token_embedding.weight : torch.Size([49408, 512])
text_model.embeddings.position_embedding.weight : torch.Size([77, 512])
text_model.encoder.layers.0.self_attn.k_proj.weight : torch.Size([512, 512])
text_model.encoder.layers.0.self_attn.k_proj.bias : torch.Size([512])
text_model.encoder.layers.0.self_attn.v_proj.weight : torch.Size([512, 512])
text_model.encoder.layers.0.self_attn.v_proj.bias : torch.Size([512])
text_model.encoder.layers.0.self_attn.q_proj.weight : torch.Size([512, 512])
text_model.encoder.layers.0.self_attn.q_proj.bias : torch.Size([512])
text_model.encoder.layers.0.self_attn.out_proj.weight : torch.Size([512, 512])
text_model.encoder.layers.0.self_attn.out_proj.bias : torch.Size([512])
text_model.encoder.layers.0.layer_norm1.weight : torch.Size([512])
text_model.encoder.layers.0.layer_norm1.bias : torch.Size([512])
text_model.encoder.layers.0.mlp.fc1.weight : torch.Size([2048, 512])
text_model.encoder.layers.0.mlp.fc

In [48]:
def convert_model_to_checkpoint(checkpoint_state_dict):
    """Convert the openai clip vision model checkpoint keys
        to vit (mae) vision model checkpoint keys
    """
    converted_state_dict = {}
    
    # delete non vision model keys
    for key in list(checkpoint_state_dict.keys()):
        if not key.startswith("vision_model"):
            del checkpoint_state_dict[key]
            continue

    # embeddings
    if "vision_model.embeddings.class_embedding" in checkpoint_state_dict:
        converted_state_dict["cls_token"] = checkpoint_state_dict.pop("vision_model.embeddings.class_embedding")
    
    if "vision_model.embeddings.position_embedding.weight" in checkpoint_state_dict:
        converted_state_dict["pos_embed"] = checkpoint_state_dict.pop("vision_model.embeddings.position_embedding.weight").unsqueeze(0)
    
    if "vision_model.embeddings.patch_embedding.weight" in checkpoint_state_dict:
        converted_state_dict["patch_embed.proj.weight"] = checkpoint_state_dict.pop("vision_model.embeddings.patch_embedding.weight")
    
    #  post_layernorm -> norm
    if "vision_model.post_layernorm.weight" in checkpoint_state_dict:
        converted_state_dict["norm.weight"] = checkpoint_state_dict.pop("vision_model.post_layernorm.weight")
    
    if "vision_model.post_layernorm.bias" in checkpoint_state_dict:
        converted_state_dict["norm.bias"] = checkpoint_state_dict.pop("vision_model.post_layernorm.bias")

    # convert vsion model keys
    for key, value in checkpoint_state_dict.items():
        new_key = key
        
        # 处理 vision_model.encoder.layers -> blocks
        if "vision_model.encoder.layers" in key:
            new_key = new_key.replace("vision_model.encoder.layers", "blocks")
        
        # 处理 self_attn -> attn
        if "self_attn" in key:
            new_key = new_key.replace("self_attn", "attn")
            
            # 处理 q_proj, k_proj, v_proj 合并为 qkv
            if "q_proj.weight" in key:
                # 假设 q_proj, k_proj, v_proj 依次出现在 state_dict 中
                q_weight = value
                k_weight = checkpoint_state_dict[key.replace("q_proj", "k_proj")]
                v_weight = checkpoint_state_dict[key.replace("q_proj", "v_proj")]
                qkv_weight = torch.cat([q_weight, k_weight, v_weight], dim=0)
                converted_state_dict[new_key.replace("q_proj.weight", "qkv.weight")] = qkv_weight
                continue
            elif "q_proj.bias" in key:
                # 假设 q_proj, k_proj, v_proj 依次出现在 state_dict 中
                q_bias = value
                k_bias = checkpoint_state_dict[key.replace("q_proj", "k_proj")]
                v_bias = checkpoint_state_dict[key.replace("q_proj", "v_proj")]
                qkv_bias = torch.cat([q_bias, k_bias, v_bias], dim=0)
                converted_state_dict[new_key.replace("q_proj.bias", "qkv.bias")] = qkv_bias
                continue
        
        # 处理 out_proj -> proj
        if "out_proj" in key:
            new_key = new_key.replace("out_proj", "proj")
        
        # 处理 layer_norm1 -> norm1, layer_norm2 -> norm2
        new_key = new_key.replace("layer_norm1", "norm1")
        new_key = new_key.replace("layer_norm2", "norm2")
        
        # 保存转换后的参数
        converted_state_dict[new_key] = value
    
    return converted_state_dict

In [49]:
new_checkpoint = convert_model_to_checkpoint(checkpoint_dict)

In [63]:
print_checkpoint(new_checkpoint)

pos_embed : torch.Size([1, 197, 768])
patch_embed.proj.weight : torch.Size([768, 3, 16, 16])
norm.weight : torch.Size([768])
norm.bias : torch.Size([768])
vision_model.pre_layrnorm.weight : torch.Size([768])
vision_model.pre_layrnorm.bias : torch.Size([768])
blocks.0.attn.k_proj.weight : torch.Size([768, 768])
blocks.0.attn.k_proj.bias : torch.Size([768])
blocks.0.attn.v_proj.weight : torch.Size([768, 768])
blocks.0.attn.v_proj.bias : torch.Size([768])
blocks.0.attn.qkv.weight : torch.Size([2304, 768])
blocks.0.attn.qkv.bias : torch.Size([2304])
blocks.0.attn.proj.weight : torch.Size([768, 768])
blocks.0.attn.proj.bias : torch.Size([768])
blocks.0.norm1.weight : torch.Size([768])
blocks.0.norm1.bias : torch.Size([768])
blocks.0.mlp.fc1.weight : torch.Size([3072, 768])
blocks.0.mlp.fc1.bias : torch.Size([3072])
blocks.0.mlp.fc2.weight : torch.Size([768, 3072])
blocks.0.mlp.fc2.bias : torch.Size([768])
blocks.0.norm2.weight : torch.Size([768])
blocks.0.norm2.bias : torch.Size([768])
bloc

In [60]:
vit_checkpoint = torch.load('/jmain02/home/J2AD007/txk47/cxz00-txk47/cliport/checkpoints/Croco_vit_base16.pth')

/tmp/ipykernel_1141508/4100341605.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vit_checkpoint = torch.load('/jmain02/home/J2AD007/txk47/cxz00-txk47/cliport/checkpoint

In [61]:
print_checkpoint(vit_checkpoint['model'])

mask_token : torch.Size([1, 1, 512])
enc_pos_embed : torch.Size([196, 768])
dec_pos_embed : torch.Size([196, 512])
patch_embed.proj.weight : torch.Size([768, 3, 16, 16])
patch_embed.proj.bias : torch.Size([768])
enc_blocks.0.norm1.weight : torch.Size([768])
enc_blocks.0.norm1.bias : torch.Size([768])
enc_blocks.0.attn.qkv.weight : torch.Size([2304, 768])
enc_blocks.0.attn.qkv.bias : torch.Size([2304])
enc_blocks.0.attn.proj.weight : torch.Size([768, 768])
enc_blocks.0.attn.proj.bias : torch.Size([768])
enc_blocks.0.norm2.weight : torch.Size([768])
enc_blocks.0.norm2.bias : torch.Size([768])
enc_blocks.0.mlp.fc1.weight : torch.Size([3072, 768])
enc_blocks.0.mlp.fc1.bias : torch.Size([3072])
enc_blocks.0.mlp.fc2.weight : torch.Size([768, 3072])
enc_blocks.0.mlp.fc2.bias : torch.Size([768])
enc_blocks.1.norm1.weight : torch.Size([768])
enc_blocks.1.norm1.bias : torch.Size([768])
enc_blocks.1.attn.qkv.weight : torch.Size([2304, 768])
enc_blocks.1.attn.qkv.bias : torch.Size([2304])
enc_bloc

In [58]:
vit_checkpoint['model'].keys()

odict_keys(['cls_token', 'pos_embed', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.proj.weight', 'blocks.2.attn.proj.bias', 'blocks.2.norm2.weight', 'blocks.2.norm2.bias', 'blocks.2.mlp.fc1.weight', 'blocks.2.mlp.fc1.bias', 'blocks.2.mlp.fc2.weight', 'blocks.2.mlp.fc2.bias', 'blocks.3.norm1.weight', 'blocks.3.norm1.bias', 'blocks.3.

In [ ]:
torch.save(new_checkpoint, '/jmain02/home/J2AD007/txk47/cxz00-txk47/cliport/checkpoints/clip_vit_base_patch16.pth')